In [ ]:
from blueshift.finance import commission, slippage
from blueshift.api import(  symbol,
                          order_target_percent,
                          set_commission,
                          set_slippage,
                          schedule_function,
                          date_rules,
                          time_rules,
                          set_stoploss,
                          set_takeprofit
                     )




def initialize(context):
  # futre index - Bank Nifty
  context.future = symbol("BANKNIFTY-I")


  # define strategy parameters
  context.params = {'indicator_lookback':3,
                    'indicator_freq':'1d',
                    'buy_signal':1,
                    'sell_signal':-1}




  # variables to track signals and target portfolio
  context.signal = 0
  context.target_position = 0




  # set trading cost to 0.3% per dollar and slippage to 0.004%
  set_commission(commission.PerShare(cost=0.003, min_trade_cost=0.0))
  set_slippage(slippage.FixedSlippage(0.00004))


  # strategy is run everyday at market open
  schedule_function(place_orders, date_rules.every_day(),
                    time_rules.market_open(minutes=0))
   # last trade can be placed at a maximum of 30 minutes before market close
  schedule_function(stop_trading, date_rules.every_day(),
                    time_rules.market_close(minutes=30))
  context.trade = True

def before_trading_start(context, data):
  context.trade = True
def stop_trading(context, data):
  context.trade = False


def place_orders(context, data):
  if not context.trade:
      return

  try:
    price_data_close = data.history(context.future, 'close',
          context.params['indicator_lookback'],
          context.params['indicator_freq'])




    price_data_open = data.history(context.future, 'open',
          context.params['indicator_lookback'],
          context.params['indicator_freq'])
  except:
      return





  px_close = price_data_close.values
  px_open = price_data_open.values
  context.signal = signal_function(px_close, px_open, context.params)

  if context.signal == context.params['buy_signal']:
    context.target_position = 1
  elif context.signal == context.params['sell_signal']:
    context.target_position= -1
  else:
    context.target_position= 0

  order_target_percent(context.future, context.target_position)
  set_takeprofit(context.future,"PERCENT",0.5)   # profit booking at 50%
  set_stoploss(context.future,"PERCENT",0.03)    # stoploss at 3%




def signal_function(px_close, px_open, params):
   # buy signals

   # gap-up opening
  if px_open[-1] > px_close[-2]:
      return 1
   # 1% price change followed by 1% gain (green candle)
  elif abs((px_close[-2] - px_open[-2])/px_open[-2]) > 0.01 and ((px_close[-1] - px_open[-1])/px_open[-1]) > 0.01:
      return 1


   # sell signals


   # price drop (red candle)
  if px_open[-1] < px_close[-1]:
      return -1
   # 1% gain (green candle) followed by a doji (indicates trend reversal)
  elif abs((px_close[-1] - px_open[-1])/px_open[-1]) < 0.001 and ((px_close[-2] - px_open[-2])/px_open[-2]) > 0.01:
      return -1


  else:
      return 0